<a href="https://colab.research.google.com/github/ecehanyildirim/Poem_Generation_WithTensorflow/blob/main/poem_generation_ipynb_adl%C4%B1_not_defterinin_kopyas%C4%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

### Import TensorFlow and other libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time

In [ ]:
text = open("siir.txt",'rb').read().decode(encoding='utf-8')
#text = open("kahramanlık.txt",'rb').read().decode(encoding='utf-8')
print(f'Length of text: {len(text)} characters')

Length of text: 686540 characters


### Read the data

First, look in the text:

In [ ]:
# Take a look at the first 250 characters in text
print(text[:250])


YAD*
Güzel günlerim vardı yağmurlarla ıslanan,
Ve güzel gecelerim masallarla dopdolu.
Her şey, her şey güzeldi, gözyaşı, dünya, zaman,
Böğürtlen topladığım ıssız, tozlu köy yolu,
Güzel günlerim vardı yağmurlarla ıslanan.
Ufacık korumuzda dola


In [ ]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

109 unique characters


## Process the text

### Vectorize the text

Before training, you need to convert the strings to a numerical representation. 

The `preprocessing.StringLookup` layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [ ]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Now create the `preprocessing.StringLookup` layer:

In [ ]:
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab))

It converts form tokens to character IDs, padding with `0`:

In [ ]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[59, 60, 61, 62, 63, 64, 65], [82, 83, 84]]>

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use `preprocessing.StringLookup(..., invert=True)`.  

Note: Here instead of passing the original vocabulary generated with `sorted(set(text))` use the `get_vocabulary()` method of the `preprocessing.StringLookup` layer so that the padding and `[UNK]` tokens are set the same way.

In [ ]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True)

This layer recovers the characters from the vectors of IDs, and returns them as a `tf.RaggedTensor` of characters:

In [ ]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

You can `tf.strings.reduce_join` to join the characters back into strings. 

In [ ]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(686540,), dtype=int64, numpy=array([ 3,  2, 53, ...,  2,  3,  2])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))




Y
A
D
*



G
ü


In [ ]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

The `batch` method lets you easily convert these individual characters to sequences of the desired size.

In [ ]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'\r' b'\n' b'Y' b'A' b'D' b'*' b'\r' b'\n' b'G' b'\xc3\xbc' b'z' b'e'
 b'l' b' ' b'g' b'\xc3\xbc' b'n' b'l' b'e' b'r' b'i' b'm' b' ' b'v' b'a'
 b'r' b'd' b'\xc4\xb1' b' ' b'y' b'a' b'\xc4\x9f' b'm' b'u' b'r' b'l' b'a'
 b'r' b'l' b'a' b' ' b'\xc4\xb1' b's' b'l' b'a' b'n' b'a' b'n' b',' b'\r'
 b'\n' b'V' b'e' b' ' b'g' b'\xc3\xbc' b'z' b'e' b'l' b' ' b'g' b'e' b'c'
 b'e' b'l' b'e' b'r' b'i' b'm' b' ' b'm' b'a' b's' b'a' b'l' b'l' b'a'
 b'r' b'l' b'a' b' ' b'd' b'o' b'p' b'd' b'o' b'l' b'u' b'.' b'\r' b'\n'
 b'H' b'e' b'r' b' ' b'\xc5\x9f' b'e' b'y' b',' b' ' b'h'], shape=(101,), dtype=string)


It's easier to see what this is doing if you join the tokens back into strings:

In [ ]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'\r\nYAD*\r\nG\xc3\xbczel g\xc3\xbcnlerim vard\xc4\xb1 ya\xc4\x9fmurlarla \xc4\xb1slanan,\r\nVe g\xc3\xbczel gecelerim masallarla dopdolu.\r\nHer \xc5\x9fey, h'
b'er \xc5\x9fey g\xc3\xbczeldi, g\xc3\xb6zya\xc5\x9f\xc4\xb1, d\xc3\xbcnya, zaman,\r\nB\xc3\xb6\xc4\x9f\xc3\xbcrtlen toplad\xc4\xb1\xc4\x9f\xc4\xb1m \xc4\xb1ss\xc4\xb1z, tozlu k\xc3\xb6y yolu,\r\nG\xc3\xbczel g\xc3\xbcnlerim v'
b'ard\xc4\xb1 ya\xc4\x9fmurlarla \xc4\xb1slanan.\r\nUfac\xc4\xb1k korumuzda dola\xc5\x9f\xc4\xb1rd\xc4\xb1m korkuyla,\r\nVe Allah\xc4\xb1 arard\xc4\xb1m ser\xc3\xa7e yuvalar\xc4\xb1nda'
b',\r\nBulamay\xc4\xb1nca dua yollard\xc4\xb1m akan suyla,\r\nG\xc3\xb6\xc4\x9f\xc3\xbc bulutlar saran bahar havalar\xc4\xb1nda,\r\nDola\xc5\x9f\xc4\xb1rd\xc4\xb1m ufac\xc4\xb1k k'
b'orumuzda korkuyla.\r\nSeyrederdim g\xc3\xb6klerde her g\xc3\xbcn b\xc3\xbcy\xc3\xbcyen ay\xc4\xb1.\r\nVe kale duvar\xc4\xb1ndan y\xc4\xb1k\xc4\xb1k mezarl\xc4\xb1klar\xc4\xb1,'


For training you'll need a dataset of `(input, label)` pairs. Where `input` and 
`label` are sequences. At each time step the input is the current character and the label is the next character. 

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'\r\nYAD*\r\nG\xc3\xbczel g\xc3\xbcnlerim vard\xc4\xb1 ya\xc4\x9fmurlarla \xc4\xb1slanan,\r\nVe g\xc3\xbczel gecelerim masallarla dopdolu.\r\nHer \xc5\x9fey, '
Target: b'\nYAD*\r\nG\xc3\xbczel g\xc3\xbcnlerim vard\xc4\xb1 ya\xc4\x9fmurlarla \xc4\xb1slanan,\r\nVe g\xc3\xbczel gecelerim masallarla dopdolu.\r\nHer \xc5\x9fey, h'


### Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Build The Model

This section defines the model as a `keras.Model` subclass (For details see [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)). 

This model has three layers:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/text_generation_training.png?raw=1)

Note: For training you could use a `keras.Sequential` model here. To  generate text later you'll need to manage the RNN's internal state. It's simpler to include the state input and output options upfront, than it is to rearrange the model architecture later. For more details see the [Keras RNN guide](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse).

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 111) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [ ]:
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  28416     
_________________________________________________________________
gru (GRU)                    multiple                  3938304   
_________________________________________________________________
dense (Dense)                multiple                  113775    
Total params: 4,080,495
Trainable params: 4,080,495
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [ ]:
sampled_indices

array([ 56,   5, 102,  11,  26,  59,  18,  20,  47,  63,  60,   8,  81,
        46,  72, 108,  44,  46,  24,  28,   4,  89,   6,  20,  20,  17,
        38,  98,   6,  38,  83,  95,  56,  13,  43,  38, 103,  73,  25,
        99,  75,   7,  26,  24,  67,  86, 102,  66,  71,  70,  93,  83,
        78,  23,  25,   7,  13,  25,  63,  62,  95,  58,  90,   9,  33,
        70,  75,  58,  58,  55,  27,  73,  93, 105,  38,  88,  46,  61,
       108,  48,  66,  37,  54,  60, 110,  75,  90,   1,  10,  50,  66,
        52,  84,  30,  11,  67,   5, 100,  31,  99])

Decode these to see the text predicted by this untrained model:

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'O g\xc3\xbcn, bir evde, o kedi\r\nBeni taa \xc3\xa7ocuklu\xc4\x9fumdan ald\xc4\xb1\r\nO g\xc3\xbcn, o evdeki, o kedi,\r\nBak-i\xc5\x9fte, neler olmu'

Next Char Predictions:
 b'\\!\xc5\x9f,;a35Seb(wRn\xe2\x80\xa2OR9> \xc3\x96"552I\xc4\x9f"Iy\xc3\xbb\\.NI\xe2\x80\x93o:\xc4\xb0q\';9i\xc2\xbb\xc5\x9fhml\xc3\xaeyt8:\'.:ed\xc3\xbb^\xc3\x9c)Dlq^^[<o\xc3\xae\xe2\x80\x99I\xc3\x87Rc\xe2\x80\xa2ThHZb\xe2\x96\xa0q\xc3\x9c[UNK]*VhXzA,i!\xc4\xb1B\xc4\xb0'


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the `from_logits` flag.


In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 100, 111)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.7098055


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [ ]:
tf.exp(mean_loss).numpy()

111.03056

Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [ ]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [ ]:
EPOCHS = 20

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
106/106 [==============================] - 8s 52ms/step - loss: 2.9373
Epoch 2/20
106/106 [==============================] - 6s 52ms/step - loss: 2.1758
Epoch 3/20
106/106 [==============================] - 6s 52ms/step - loss: 2.0254
Epoch 4/20
106/106 [==============================] - 6s 53ms/step - loss: 1.9058
Epoch 5/20
106/106 [==============================] - 6s 53ms/step - loss: 1.7964
Epoch 6/20
106/106 [==============================] - 6s 53ms/step - loss: 1.7021
Epoch 7/20
106/106 [==============================] - 6s 54ms/step - loss: 1.6196
Epoch 8/20
106/106 [==============================] - 6s 54ms/step - loss: 1.5485
Epoch 9/20
106/106 [==============================] - 6s 54ms/step - loss: 1.4811
Epoch 10/20
106/106 [==============================] - 6s 55ms/step - loss: 1.4160
Epoch 11/20
106/106 [==============================] - 6s 55ms/step - loss: 1.3507
Epoch 12/20
106/106 [==============================] - 7s 56ms/step - loss: 1.2830
Epoch 13/20
1

## Generate text

The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.

![To generate text the model's output is fed back to the input](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/text_generation_sampling.png?raw=1)

Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.


The following makes a single step prediction:

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "" or "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['', '[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "" or "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Run it in a loop to generate some text. Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['ŞAİR:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*400)
print('\nRun time:', end - start)

ŞAİR:
Dört kendine oldum bir kilise, bir gün
Bir gün, bir damar aynı akşamına.
Kuşlar uçurum, kan?IDIr elbet
Kesin daha
KALI OVURUM
gün sabahlanılmıştır telâşınü alanıyor, tiren olubüslerini
Yakma çiçeklerin açlığı
oy farfara farfara
ateşlerinde parşai
elimdeki otel bir yere gideri
dönüyorum yatağım terseteli
büyük açaklan herkesin
kutsal kirazlarına, motorlara yüzükleri
yıllarca soğra geçirmez bir isyandı, can çol uzar, oynanı
toprak adamları boğazdan gitmenin yoktu
ziirapı özleyecektir başkadır
tutmakları tutturmalıydık kalmasından
sezmektir
kimbilir bir dateğit doğa'ya götürüyordu
denizden ona osmanlığında tam da tutan gelir
yarı gecesi değişerek
elimde de şimdilik
şarkılar söylemez
nerden baksa/
 ama belleğinde herhangi bir duvar
ben utançına sinmişliğin vazgeçemlerinin aldı belir
kara gecen ne kadar küçük
ortalık gemileri sanı olduk çeşti duvarların
bir çelişin eli yola çıkınır olduğu yerde
kim buldu ki bemaz elbit hiç öfte iyidir
kaldırımızı kazdır insan 

_______________________

The easiest thing you can do to improve the results is to train it for longer (try `EPOCHS = 30`).

You can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.

If you want the model to generate text *faster* the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above. 

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['sair:', 'sair:', 'sair:', 'sair:', 'sair:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\r\nuykusuzluk ve m\xc3\xbckezvr\xc5\x9fu vazdim\r\nkadehimi k\xc4\xb1r\xc4\xb1yorlar\r\nhasil kalanyad\xc4\xb1r.\r\nKimi adlar de\xc4\x9fildi onlardan\r\nMahzun bir h\xc3\xbcz\xc3\xbcn verir.\r\nSapsar\xc4\xb1 s\xc3\xbccahlardad\xc4\xb1r.\r\nSOKA\xc5\x9e SOLDAYA, BUYUKLAR DURDAR\r\nBana b\xc3\xb6lerler bir insan geri G\xc3\xb6rd\xc3\xbc\xc4\x9f\xc3\xbcm sevgilim\r\nbu i\xc5\x9fimizi besleyince ans\xc4\xb1z\xc4\xb1n k\xc3\xb6t\xc3\xbcl\xc3\xbcklerini\r\ns\xc3\xb6yle bizi k\xc3\xb6t\xc3\xbcnde biliyorsun\r\n-Garson rak\xc4\xb1 benzeyebilir.\r\nHer \xc5\x9fey i\xc5\x9fte b\xc3\xb6yle oldu, \xc3\xb6l\xc3\xbcre yakt\xc4\xb1m.\r\nOysa Kelemes kalans\xc4\xb1n\xc4\xb1z.\r\nBir ba\xc5\x9fka hi\xc3\xa7 olan\xc4\xb1n en g\xc3\xbczel sesleri.\r\nKINA TU\xc5\x9e'n'n bir g\xc3\xbcn sabah vakti bitirdim.\r\nYERMEL\xc4\xb0M\r\nS\xc3\xb6ylendimiziyor,, ey s\xc3\xb6z\xc3\xbc, bir g\xc3\xb6n b\xc3\xbcy\xc3\xbck ve kanlar\r\nKalk\xc4\xb1l\xc4\xb1lm\xc4\xb1\xc5\x9f bir yalan h\xc3\xbcz

## Export the generator

This single-step model can easily be [saved and restored](https://www.tensorflow.org/guide/saved_model), allowing you to use it anywhere a `tf.saved_model` is accepted.

In [ ]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')


FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.


INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


In [ ]:
states = None
next_char = tf.constant(['ŞAİR:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ŞAİR:
Sönük bir salınan
Gül kurusu casde...
Karanlıkta öğle yalnızlığım bir adam için yoksa, bir fican


## Gelişmiş: Özelleştirilmiş Eğitim

Yukarıdaki eğitim prosedürü basittir, ancak size fazla kontrol sağlamaz.
Kötü tahminlerin modele geri beslenmesini önleyen öğretmen zorlamasını kullanır, böylece model asla hatalardan kurtulmayı öğrenmez.

Artık modeli manuel olarak nasıl çalıştıracağınızı gördüğünüze göre, eğitim döngüsünü uygulayacaksınız. Bu, örneğin, modelin açık döngü çıktısını dengelemeye yardımcı olmak için _müfredat öğrenimi_ uygulamak istiyorsanız bir başlangıç noktası sağlar.

Özel bir eğitim döngüsünün en önemli kısmı, tren adımı işlevidir.

In [ ]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [ ]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [ ]:
model.fit(dataset, epochs=1)

106/106 [==============================] - 8s 55ms/step - loss: 2.9342


Or if you need more control, you can write your own complete custom training loop:

In [ ]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.2903
Epoch 1 Batch 50 Loss 2.1710
Epoch 1 Batch 100 Loss 2.1250

Epoch 1 Loss: 2.1680
Time taken for 1 epoch 6.85 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 2.0554
Epoch 2 Batch 50 Loss 2.0279
Epoch 2 Batch 100 Loss 1.9413

Epoch 2 Loss: 2.0219
Time taken for 1 epoch 10.24 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.9597
Epoch 3 Batch 50 Loss 1.8814
Epoch 3 Batch 100 Loss 1.9033

Epoch 3 Loss: 1.9042
Time taken for 1 epoch 6.39 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.8348
Epoch 4 Batch 50 Loss 1.8214
Epoch 4 Batch 100 Loss 1.8050

Epoch 4 Loss: 1.7975
Time taken for 1 epoch 6.33 sec
________________________________________________________________________________
Epoch 5 Batch 0 Loss 1.7254
Epoch 5 Batch 50 Loss 1.7436
Epoch 5 Batch 100 Loss 1.6629

Epoch 5 Loss: 1.7048
Ti

In [ ]:
start = time.time()
states = None
#next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

n
Sevdalı yaşanırlardı gidip gelmelerde. Ardına kadar
Turnam ben söndüğünü görmemiş.
Fakirliğiye, bir nice seni,
halde bile beyazlarında kavga çok geldi
-suy hartmiz masa, dal bir şeyler haçıldı, bilmedim yazdıklarım:
bütün çiçeklerle tutkusuzlukla değişir
harcunç çekirse de evrensel yetişiriz,
sonsuz girişim kendi derdi ye yakış
ya bir bu bu akin
neya hambettiğimiz
sen göğsünde bunların üstünden
öldürül esirgiçemizleri
rüzgârın eskidirlerin.
zaten yememişlere alışkın öbürleriyle sulara göke lina denge
Korkunun ilk yapılır kralin yolundaydı
Babamın ağladığını duyuyorlar, bildikleri, getirdim.
SESSİN YER BEYLEDİ
Oysa k o k de ki, Varda durdu Herkes
Yürek geceye darmadağın
Vapurlarım dökünün o dilin kullanından
Bir o redi benden nedir.
Gözlerinde yazması...
Siysiler, bel, gemiciler,
Fasmanını kovalamaktı.
Kuru bir sözüm, kuruyorum canıma
IIÇtılar birden başladı sarmıyordum
gülluğuma içinde herhalde ve gökten korkar'sa
ben alınız al müzüm ki unutulmaz bir deniz  

________________________

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['Şair:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Şair:
ÇaĞRI GÜLÜN BİRİ GELİŞ
Dursuz düzeninde çanta kaybolun
Güley gerekesi...
Bu evlerini akıyordu. Gözlerinde
Biz bu harladığımı gördük. Kim tek tek bitti
Hey gözüne katmış bir
lerzet her aklımda:
o kedi gözleriyle yedilenle kederse, bir dağın yağmuru yanılmadın, telcik olur.
Kimbilir kaç kere öpsem ikinin hatırı için
Akşam akşam yanmıştım, o levlik.
Babam, Yani Sül burada kalsın..
Bekleşmeyen düzdanın yılmanın yeniden başla hen
bir kaptan ki artık elleridir.
Göl yaban, tay büyütürsünü, bir kadınla bir şeyler yorgun gemilere, hey gidi.
Gelir durmuş; Paslatacaksın. Dünoyla beni yeni bir adam, köyünden
En uzun, bir karıştı, jamparaları durgunluktur
Bir geceyi bir kadından korkur yolcularıydı..
Bir durup bütün bunlar bütün yalnızlığın adını...
HIZLAN DÖMÜDA
Şimdi bir güvercinin
Şu dönleri görmedim, Sarhoşuo
Kent ey sevişin, yoktursa, daha da esmer içindeydi.
Basınmalar güzelde, bundan,
Bana Böneş kesip udanıyorum ki
Beni şaşırdım durdum.
Kendi divalarla
O güneş vur 

___________________